<a href="https://colab.research.google.com/github/MrFlexi/LegoPredict/blob/master/Lego_TrainVGG16_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/MrFlexi/LegoPredict.git

!ls

from google.colab import drive
drive.mount('/content/gdrive')


# Tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


Cloning into 'LegoPredict'...
remote: Enumerating objects: 4189, done.
remote: Total 4189 (delta 0), reused 0 (delta 0), pack-reused 4189
Receiving objects: 100% (4189/4189), 44.36 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (507/507), done.
LegoPredict  sample_data
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
--2020-04-21 07:33:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.232.135.84, 52.3.79.57, 34.197.27.35, ...
Connecting 

In [2]:
# Tensorboard get URL 
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://8e853e8c.ngrok.io


In [5]:



import_flowers = 'No' #@param ["No", "Yes"]
image_sources  = 'lego_fotos' #@param {type:"string"} 
image_size = 224 #@param {type:"integer"} 
number_classes = 6 #@param {type:"integer"} 
number_epochs =   2#@param {type:"integer"} 
save_model_path = 'LegoTrainedVGG16_15Layer_classes6_epochs_40.h5' #@param {type:"string"}
best_model_path = 'LegoTrainedVGG16_15Layer_classes6_best_model.h5' #@param {type:"string"} 
labels_file     = 'labels_classes6.json' #@param {type:"string"} 
train_batchsize = 50 #@param {type:"integer"} 
val_batchsize = 50 #@param {type:"integer"} 




import os   # set working directory to 
import json
os.chdir("/content/LegoPredict")
!ls
#!wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.0/resnet50_coco_best_v2.1.0.h5  -P /content/keras-retinanet/snapshots/


if import_flowers == "Yes":
    os.chdir("/content/LegoPredict/lego_fotos/train")
    !wget http://download.tensorflow.org/example_images/flower_photos.tgz 
    !tar -xf flower_photos.tgz
    os.chdir("/content/LegoPredict/")
    !cp -r ./lego_fotos/train/flower_photos/daisy/ ./lego_fotos/train/
    !cp -r ./lego_fotos/train/flower_photos/dandelion/ ./lego_fotos/train/
    !cp -r ./lego_fotos/train/flower_photos/roses/ ./lego_fotos/train/
    !cp -r ./lego_fotos/train/flower_photos/sunflowers/ ./lego_fotos/train/
    !cp -r ./lego_fotos/train/flower_photos/tulips/ ./lego_fotos/train/
    !cp -r ./lego_fotos/train/flower_photos/daisy/ ./lego_fotos/validation/
    !cp -r ./lego_fotos/train/flower_photos/dandelion/ ./lego_fotos/validation/
    !cp -r ./lego_fotos/train/flower_photos/roses/ ./lego_fotos/validation/
    !cp -r ./lego_fotos/train/flower_photos/sunflowers/ ./lego_fotos/validation/
    !cp -r ./lego_fotos/train/flower_photos/tulips/ ./lego_fotos/validation/
    !rm -r ./lego_fotos/train/flower_photos/
    !rm ./lego_fotos/train/flower_photos.tgz
    os.chdir("/content/LegoPredict")


CamMarkFeature.py      lego_fotos	      settings_dialog.py
candle.jpg	       models		      settings.json
CaptureImageWebcam.py  PredictBatch.py	      test.py
fineTuning4.py	       PredictFromFolder2.py  test_txt.py
fineTuning.py	       PredictFromFolder.py   TrainInceptionV3.py
icon.gif	       PredictFromWebcam.py   TrainMobilnet_Transfer.py
labels.json	       Predict.py	      UseFinetunedLego.py
lables.txt	       result.json	      UsePrettrainedApp.py


In [0]:
#!cp /content/gdrive/My*Drive/keras-retinanet/images/* /content/keras-retinanet/images/

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
import os
import time

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

callbacks= [ReduceLROnPlateau(monitor='val_acc', 
                                        patience=5, 
                                        verbose=1, 
                                        factor=0.5, 
                                        min_lr=0.00001),
            
           TensorBoard(log_dir='./log', histogram_freq=0,
                         write_graph=True,
                         write_grads=True,
                         batch_size=train_batchsize,
                         write_images=True),
            
           EarlyStopping(monitor='val_loss', patience=10 ),
           ModelCheckpoint(filepath=best_model_path, monitor='val_loss', save_best_only=True)
           ]

train_dir = "./" + image_sources + "/train"
validation_dir = "./" + image_sources + "/validation"
prediction_dir = "./" + image_sources + "/predict"


# Change the batchsize according to your system RAM


from keras.applications import VGG16

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

# Freeze all the layers
for layer in vgg_conv.layers[:-15]:
    layer.trainable = True



from keras import models
from keras import layers
from keras import optimizers

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(number_classes, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
#model.summary()

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)


#  TRAIN
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)



# Data Generator for Training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')

# Get the label to class mapping from the generator
label2index = train_generator.class_indices

# Getting the mapping from class index to class label
idx2label = dict((v, k) for k, v in label2index.items())

print (idx2label)
with open(labels_file, 'w') as fp:
    json.dump(idx2label, fp)



# Data Generator for Validation data
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train the Model
# NOTE that we have multiplied the steps_per_epoch by 2. This is because we are using data augmentation.
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2*train_generator.samples/train_generator.batch_size ,
      epochs=number_epochs,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,
      callbacks = callbacks)

# Save the Model
model.save(save_model_path)

!cp /content/LegoPredict/*.h5 "/content/gdrive/My Drive/keras-retinanet/snapshots/"

# Plot the accuracy and loss curves
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


#----------------------------------------------------------------
#       Predict
#----------------------------------------------------------------

# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
    prediction_dir,
    target_size=(image_size, image_size),
    batch_size=val_batchsize,
    class_mode='categorical',
    shuffle=False)

# Get the filenames from the generator
fnames = validation_generator.filenames

# Get the ground truth from generator
ground_truth = validation_generator.classes

# Get the label to class mapping from the generator
label2index = validation_generator.class_indices

# Getting the mapping from class index to class label
idx2label = dict((v, k) for k, v in label2index.items())

# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator,
                                      steps=validation_generator.samples / validation_generator.batch_size, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors), validation_generator.samples))

# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]

    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])

    original = load_img('{}/{}'.format(prediction_dir, fnames[errors[i]]))
    plt.figure(figsize=[7, 7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

AttributeError: ignored

In [0]:
!cp /content/LegoPredict/*.h5 "/content/gdrive/My Drive/keras-retinanet/snapshots/"

In [0]:
# https://engmrk.com/kerasapplication-pre-trained-model/

save_model_path = 'LegoTrainedVGG16_15Layer_classes6_best_model.h5' #@param {type:"string"} 

import keras
import numpy as np

from keras.applications import inception_v3
from keras.models 		import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from keras.preprocessing.image import ImageDataGenerator, load_img

inception_model = inception_v3.InceptionV3(weights="imagenet")





#from pygments.lexer import include

#os.environ["HTTPS_PROXY"] = "http://proxy.le.grp:8080"

def read_model():
	
	print(model.summary())
	return model


def show_results():
    # Show the results
    for i in range(len(predicted_classes)):
        pred_class = np.argmax(predictions[predicted_classes[i]])
        pred_label = idx2label[pred_class]

        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            fnames[predicted_classes[i]].split('/')[0],
            pred_label,
            predictions[predicted_classes[i]][pred_class])

        original = load_img('{}/{}'.format(validation_dir, fnames[predicted_classes[i]]))
        plt.figure(figsize=[7, 7])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()



def get_pil_image(filename):
    # load an image in PIL format
    original_image = load_img(filename, target_size=(224, 224))

    # convert the PIL image (width, height) to a NumPy array (height, width, channel)
    numpy_image = img_to_array(original_image)

    # Convert the image into 4D Tensor (samples, height, width, channels) by adding an extra dimension to the axis 0.
    input_image = np.expand_dims(numpy_image, axis=0)

    print("PIL image size = ", original_image.size)
    print("NumPy image size = ", numpy_image.shape)
    print("Input image size = ", input_image.shape)
    plt.imshow(np.uint8(input_image[0]))
    return input_image



def predict_inception_v3( input_image ):
    # Load the Inception_V3 model


    # preprocess for inception_v3
    processed_image_inception_v3 = inception_v3.preprocess_input(input_image.copy())

    # inception_v3
    start = time.time()
    predictions_inception_v3 = inception_model.predict(processed_image_inception_v3)
    label_inception_v3 = decode_predictions(predictions_inception_v3)
    print("label_inception_v3 = ", label_inception_v3)
    ende = time.time()
    print('{:5.3f}s'.format(ende - start))

def predict_mymodel( img_path ):

    img = image.load_img(img_path, target_size=(224, 224))
    img.show()
    print(type(img))
    print(img.format)
    print(img.mode)
    print(img.size)
    img_tensor = image.img_to_array(img)  # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor,axis=0)  # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.  # imshow expects values in the range [0, 1]
    

    start = time.time()
    predictions_my = vgg16_model.predict(img_tensor)
    #print(predictions_my)
    # label_MY = decode_predictions(predictions_my)
    # print("My labels = ", label_MY )
    ende = time.time()
    y_classes = predictions_my.argmax(axis=-1)
    print('Best Match', labels[y_classes[0]],predictions_my[0,y_classes[0]],'{:5.3f}s'.format(ende - start))


# convert JSON key field string to DICT Integer
def jsonKeys2int(x):
      if isinstance(x, dict):
        return {int(k):v for k,v in x.items()}
      return x

if __name__ == '__main__':
  
    save_model_path = "./" + save_model_path

    # load model
    vgg16_model = load_model(save_model_path)
    
    # load labels
    labels_file = "./" + labels_file
    with open(labels_file) as f:
      labels = json.load(f,object_hook=jsonKeys2int)
    print(labels)    
    
    

    
    filename_3x5LGreen = "./lego_fotos/predict/3x5LGreen/frame20190226-213256.jpg"
    filename_3x5LGreen_Band = "./lego_fotos/predict/3x5LGreen/3x5LGreen_Band.jpg"
    filename_gear = "./lego_fotos/train/Gear20Beige/frame20190222-16421620.jpg"
  

    
    filename_3x5LOrange = "./lego_fotos/predict/3x5LOrange/frame20190226-213949101.jpg"
    
    filename_3x5LRed =  "./lego_fotos/train/3x5LRed/frame20190222-1629518.jpg" 
    
    filename_1x4LRed = "./lego_fotos/train/1x4LRed/frame20190204-1642049.jpg"
     
 
    
    filename_sunflower = "./lego_fotos/unsorted/sunflower.jpg"
  


    #---------------------------------------------------------------------------------------------
    # Predict SingleImage via Inception V3
    #---------------------------------------------------------------------------------------------

    #print("Inception - Lego Brick ")
    #predict_inception_v3( lego_brick )
    #print("Inception - Lego Connector Black ")
    #predict_inception_v3( image_rose )


    #---------------------------------------------------------------------------------------------
    # My Trained Model - Single Image
    #---------------------------------------------------------------------------------------------
    # processed_image_my = my_model.preprocess_input(input_image.copy())

    print("VGG16 - 3x5LGreen ")
    predict_mymodel(filename_3x5LGreen)
    
    
    print("VGG16 - 3x5LGreen Band ")
    predict_mymodel(filename_3x5LGreen_Band)
    
    
    print("VGG16 - Gear ")
    predict_mymodel(filename_gear)
    

    print("VGG16-  3x5LOrange ")
    predict_mymodel(filename_3x5LOrange)
    
    print("VGG16--  3x5LRed ")
    predict_mymodel(filename_3x5LRed)
   
    
    
    print("VGG16-  Sunflower ")
    predict_mymodel(filename_sunflower)


    





{0: '3x5LBlack', 1: '3x5LGreen', 2: '3x5LOrange', 3: '3x5LRed', 4: 'BeltGray', 5: 'Gear20Beige'}
VGG16 - 3x5LGreen 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match 3x5LGreen 1.0 7.032s
VGG16 - 3x5LGreen Band 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match 3x5LGreen 1.0 0.014s
VGG16 - Gear 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match Gear20Beige 1.0 0.011s
VGG16-  3x5LOrange 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match 3x5LOrange 1.0 0.011s
VGG16--  3x5LRed 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match 3x5LRed 1.0 0.011s
VGG16-  Sunflower 
<class 'PIL.Image.Image'>
None
RGB
(224, 224)
Best Match 3x5LOrange 1.0 0.011s


In [0]:
!ls

CamMarkFeature.py				 PredictBatch.py
candle.jpg					 PredictFromFolder2.py
CaptureImageWebcam.py				 PredictFromFolder.py
fineTuning4.py					 PredictFromWebcam.py
fineTuning.py					 Predict.py
icon.gif					 result.json
labels_classes6.json				 settings_dialog.py
labels.json					 settings.json
lables.txt					 test.py
lego_fotos					 test_txt.py
LegoTrainedVGG16_15Layer_classes6_best_model.h5  TrainInceptionV3.py
LegoTrainedVGG16_15Layer_classes6_epochs_10.h5	 TrainMobilnet_Transfer.py
LegoTrainedVGG16_15Layer_classes6_epochs_40.h5	 UseFinetunedLego.py
log						 UsePrettrainedApp.py
models


In [0]:
# Concert Model into TF Lite Model

import tensorflow as tf
os.chdir("/content/LegoPredict")

save_model_path = 'LegoTrainedVGG16_15Layer_classes6_best_model.h5' #@param {type:"string"} 

model = load_model(save_model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

AttributeError: ignored